<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze-.iframe.experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from IPython.display import IFrame

IFrame(src='https://voodoohop.github.io/colabasaservice/', width=700, height=600)

In [ ]:
!pip install deep-daze stomp.py --upgrade

In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 

flatten = itertools.chain.from_iterable



DEFAULT_PARAMS='--lower_bound_cutout 0.4 --deeper --image-width=384 --iterations=500 --epochs=2 --save-every=15 --save-progress=True --open_folder=False --overwrite=True --save_video=True "A perfect dinner that came alive and started drawing itself."'

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)



def run(params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  !rm *.jpg
  !rm *.png

  cmd = f'imagine {params}'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  FOUND_MEDIA_FILES=[]
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    conn.send(body=line.decode("utf-8"), destination='/topic/colabOut')
    NEW_FOUND_MEDIA_FILES = get_matching_files(watch_media_paths)
    NEW_FOUND_MEDIA_FILES.sort(key=os.path.getmtime)
    ADDED_MEDIA_FILES = list(set(NEW_FOUND_MEDIA_FILES)-set(FOUND_MEDIA_FILES))
    FOUND_MEDIA_FILES = NEW_FOUND_MEDIA_FILES

    if len(ADDED_MEDIA_FILES) > 0:
      for added in ADDED_MEDIA_FILES:
        sleep(0.1)
        if len(added) > 0:
          print("reading", added)
          with open(added,"rb") as media_file:
            added_media = media_file.read()
            mimetype = guess_type(added)
            base64_encoded_added = f"data:{mimetype};base64,{b64encode(added_media).decode('ascii')}"
          
          conn.send(body=base64_encoded_added, destination='/topic/colabMediaOut')

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))
run()
conn.disconnect()



epochs:   0%|          | 0/2 [00:00<?, ?it/s]
                                             
reading ./A_perfect_dinner_that_came_alive_and_started_drawing_itself..jpg
loss: -19.51:   0%|          | 2/500 [00:05<14:38,  1.76s/it]


In [ ]:
%%javascript
// Create the channel object
const channel = new BroadcastChannel('my-channel');

// Set up a listener
channel.onmessage = message => console.log('Received:', message.data);

// Post a message to all subscribers
channel.postMessage('Hello, world');